<h1>Pre-Trained Model InceptionResNetV2, enlarged training dataset</h1>

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [2]:
batch_size_training = 100
batch_size_validation = 100

In [3]:
image_resize = 224
num_classes = 2

In [4]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [5]:
train_generator = data_generator.flow_from_directory(
    'horse2zebra/training',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 3899 images belonging to 2 classes.


In [6]:
validation_generator = data_generator.flow_from_directory(
    'horse2zebra/validation',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 513 images belonging to 2 classes.


In [7]:
model = Sequential()

In [8]:
model.add(InceptionResNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [9]:
model.add(Dense(num_classes, activation='softmax'))
model.layers

In [10]:
model.layers[0].layers

In [11]:
model.layers[0].trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dense (Dense)                (None, 2)                 3074      
Total params: 54,339,810
Trainable params: 3,074
Non-trainable params: 54,336,736
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [14]:
#fit_history = model.fit_generator(
fit_history = model.fit(    
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
39/39 [==============================] - 296s 7s/step - loss: 0.1006 - accuracy: 0.9590 - val_loss: 0.0614 - val_accuracy: 0.9864
Epoch 2/2
39/39 [==============================] - 289s 7s/step - loss: 0.0290 - accuracy: 0.9920 - val_loss: 0.0561 - val_accuracy: 0.9864


In [15]:
model.save('classifier_inception_resnet_v2_model_1.h5')

In [16]:
test_generator = data_generator.flow_from_directory(
    'horse2zebra/testing',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 742 images belonging to 2 classes.


In [18]:
scores = model.evaluate_generator(test_generator,len(test_generator)) 
print("Loss = ", scores[0])
print("Accuracy = ", scores[1])
scores_inception_resnet_v2_enlarged = scores

Loss =  0.014948882162570953
Accuracy =  0.993261456489563


In [19]:
model_inception_resnet_v2 = tensorflow.keras.models.load_model('classifier_inception_resnet_v2_model.h5')

In [20]:
scores = model_inception_resnet_v2.evaluate_generator(test_generator,len(test_generator)) 
print("Loss = ", scores[0])
print("Accuracy = ", scores[1])
scores_inception_resnet_v2 = scores

Loss =  0.016801562160253525
Accuracy =  0.993261456489563


In [21]:
loss = list()

loss.append(scores_inception_resnet_v2[0])
loss.append(scores_inception_resnet_v2_enlarged[0])

accuracy = list()

accuracy.append(scores_inception_resnet_v2[1])
accuracy.append(scores_inception_resnet_v2_enlarged[1])

labels = ['inception_resnet_v2 (training examples = 2401)', 'inception_resnet_v2 (training examples = 3899)']

In [22]:
import pandas as pd
data = {'Model':labels, 'Loss':loss, 'Accuracy':accuracy}
test_df = pd.DataFrame(data)
test_df

,Model,Loss,Accuracy
0,inception_resnet_v2 (training examples = 2401),0.016802,0.993261
1,inception_resnet_v2 (training examples = 3899),0.014949,0.993261


| Model                                       	| Loss 	   | Accuracy |
|-----------------------------------------------|----------|----------|
|inception_resnet_v2 (training examples = 2401) | 0.016802 | 0.993261 |
|inception_resnet_v2 (training examples = 3899) | 0.014949 | 0.993261 |